In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/MyDrive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM"


def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)



create_and_set_working_directory(project_folder)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM , Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import  mean_squared_error, r2_score, mean_absolute_error, max_error
import keras_tuner as kt
import tensorflow as tf 
from tensorflow import keras
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from sklearn.metrics import classification_report




/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def read_data (path):
  return pd.read_csv(path, index_col= 0 , header= 0)

In [ ]:
# load the dataset lead time 15 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/15min/Data2013.csv')

In [ ]:
# load the dataset lead time 30 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/30min/Data2013.csv')

In [ ]:
# load the dataset lead time 45 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/45min/Data2013.csv')

In [ ]:
# load the dataset lead time 60 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/60min/Data2013.csv')

In [ ]:
# load the dataset lead time 75 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/75min/Data2013.csv')

In [ ]:
# load the dataset lead time 90 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/90min/Data2013.csv')

In [ ]:
# load the dataset lead time 105 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/105min/Data2013.csv')

In [ ]:
# load the dataset lead time 120 min 
dataframe = read_data ('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/120min/Data201.csv')

In [ ]:
#Some explaratory data analysis
dataframe.head(10)

,yh1,yh2,yh3,yh4,yh5,yh6,yh7,yh8,yh9,yh10,yh11,yh12,ym1,ym2,ym3,ym4,ym5,ym6,ym7,ym8,ym9,ym10,ym11,ym12,yn1,yn2,yn3,yn4,yn5,yn6,yn7,yn8,yn9,yn10,yn11,yn12,ys1,ys2,ys3,ys4,ys5,ys6,ys7,ys8,ys9,ys10,ys11,ys12,yo,label
1,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,7.8,0.2,0.0,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,209.590,209.613,209.593,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.18455,172.18855,172.21955,172.60854,low
2,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,7.8,0.2,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,209.590,209.613,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.18455,172.18855,172.72354,low
3,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,7.8,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,209.590,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.18455,172.82254,high
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.87754,high
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,209.548,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.87854,high
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,209.548,209.548,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.86155,high
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,209.550,209.548,209.548,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.76355,high
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,209.550,209.550,209.548,209.548,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.21955,low
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209.549,209.550,209.550,209.548,209.548,209.549,209.535,209.534,209.533,209.530,209.534,209.532,172.17255,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.18855,low
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209.547,209.549,209.550,209.550,209.548,209.548,209.549,209.535,209.534,209.533,209.530,209.534,172.17255,172.17255,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.18455,low


In [ ]:
dataframe.tail(10)

,yh1,yh2,yh3,yh4,yh5,yh6,yh7,yh8,yh9,yh10,yh11,yh12,ym1,ym2,ym3,ym4,ym5,ym6,ym7,ym8,ym9,ym10,ym11,ym12,yn1,yn2,yn3,yn4,yn5,yn6,yn7,yn8,yn9,yn10,yn11,yn12,ys1,ys2,ys3,ys4,ys5,ys6,ys7,ys8,ys9,ys10,ys11,ys12,yo,label
21332,0.0,0.0,0.0,0.0,0.4,0.6,0.8,0.4,0.6,0.2,0.4,0.4,0.0,0.0,0.0,0.0,0.2,0.4,0.6,0.0,0.4,0.6,0.4,0.8,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,209.59862,209.59965,209.60167,209.60670,209.60972,209.61275,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.25035,172.24935,172.24935,172.25134,172.25634,172.27784,172.68333,low
21333,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.8,0.4,0.6,0.2,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,0.0,0.4,0.6,0.4,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,209.59862,209.59965,209.60167,209.60670,209.60972,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.25035,172.24935,172.24935,172.25134,172.25634,172.61683,low
21334,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.8,0.4,0.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,0.0,0.4,0.6,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,209.59862,209.59965,209.60167,209.60670,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.25035,172.24935,172.24935,172.25134,172.55533,low
21335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.8,0.4,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,0.0,0.4,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,209.59862,209.59965,209.60167,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.25035,172.24935,172.24935,172.48683,low
21336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.8,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,0.0,209.60237,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,209.59862,209.59965,172.25737,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.25035,172.24935,172.40933,low
21337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.6,209.60334,209.60237,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,209.59862,172.25737,172.25737,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.25035,172.35384,low
21338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,209.60631,209.60334,209.60237,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,209.59860,172.25738,172.25737,172.25737,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.25035,172.31434,low
21339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,209.60529,209.60631,209.60334,209.60237,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,209.59657,172.25838,172.25738,172.25737,172.25737,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25036,172.27784,low
21340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209.60726,209.60529,209.60631,209.60334,209.60237,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,209.59755,172.25838,172.25838,172.25738,172.25737,172.25737,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25136,172.25634,low
21341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,209.60824,209.60726,209.60529,209.60631,209.60334,209.60237,209.60139,209.59942,209.60244,209.60147,209.60049,209.59852,172.25938,172.25838,172.25838,172.25738,172.25737,172.25737,172.25687,172.25637,172.25587,172.25436,172.25336,172.25236,172.25134,low


In [ ]:
def preprocess_data(data):
  data.drop( ['yo'], inplace= True, axis = 1)
  data['output' ] = data['label']. apply (lambda  x: 0 if x == 'low' else 1)
  data.drop( ['label'], inplace= True, axis = 1)
  return data


In [ ]:
dataframe = preprocess_data(dataframe)

In [ ]:
dataframe.head()

,yh1,yh2,yh3,yh4,yh5,yh6,yh7,yh8,yh9,yh10,yh11,yh12,ym1,ym2,ym3,ym4,ym5,ym6,ym7,ym8,ym9,ym10,ym11,ym12,yn1,yn2,yn3,yn4,yn5,yn6,yn7,yn8,yn9,yn10,yn11,yn12,ys1,ys2,ys3,ys4,ys5,ys6,ys7,ys8,ys9,ys10,ys11,ys12,output
1,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,7.8,0.2,0.0,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,209.590,209.613,209.593,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.18455,172.18855,172.21955,0
2,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,7.8,0.2,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,209.590,209.613,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.18455,172.18855,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,7.8,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,209.590,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,172.18455,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,3.4,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,209.563,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,172.17655,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.4,0.0,1.4,209.548,209.549,209.535,209.534,209.533,209.530,209.534,209.532,209.529,209.539,209.544,209.550,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17255,172.17155,172.17155,172.17155,172.17155,172.17255,1


In [ ]:
dataframe.columns

Index(['yh1', 'yh2', 'yh3', 'yh4', 'yh5', 'yh6', 'yh7', 'yh8', 'yh9', 'yh10',
       'yh11', 'yh12', 'ym1', 'ym2', 'ym3', 'ym4', 'ym5', 'ym6', 'ym7', 'ym8',
       'ym9', 'ym10', 'ym11', 'ym12', 'yn1', 'yn2', 'yn3', 'yn4', 'yn5', 'yn6',
       'yn7', 'yn8', 'yn9', 'yn10', 'yn11', 'yn12', 'ys1', 'ys2', 'ys3', 'ys4',
       'ys5', 'ys6', 'ys7', 'ys8', 'ys9', 'ys10', 'ys11', 'ys12', 'output'],
      dtype='object')

In [ ]:
dataset = dataframe.values

In [ ]:
dataset

array([[  0.     ,   0.     ,   0.     , ..., 172.18855, 172.21955,
          0.     ],
       [  0.     ,   0.     ,   0.     , ..., 172.18455, 172.18855,
          0.     ],
       [  0.     ,   0.     ,   0.     , ..., 172.17655, 172.18455,
          1.     ],
       ...,
       [  0.     ,   0.     ,   0.     , ..., 172.25136, 172.25036,
          0.     ],
       [  0.     ,   0.     ,   0.     , ..., 172.25236, 172.25136,
          0.     ],
       [  0.     ,   0.     ,   0.     , ..., 172.25336, 172.25236,
          0.     ]])

In [ ]:
Input = dataset[:, :-1]
output = dataset[:, -1]

In [ ]:
# Train Test split 
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
test_X, test_y = Input[:test_size, :], output[ :test_size]
train_X, train_y=Input[test_size:, :], output[ test_size:]

In [ ]:
#Normalization
scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [ ]:
# Reshaping data in the format (number of samples, number of steps, number of features)
def reshape (multiarray): 
  train_X_2 = multiarray.reshape((multiarray.shape[0], 4, 12))
  reshaped = np.empty([multiarray.shape[0], 12, 4])
  for i in (range(multiarray.shape[0])): 
    reshaped[i, : ,  : ] = train_X_2[i, :, :].transpose()
  return (reshaped)

In [ ]:
train_X = reshape (train_X )
test_X = reshape (test_X )

In [ ]:
train_X[0,:,:]

array([[0.        , 0.02564103, 0.04444971, 0.1775494 ],
       [0.        , 0.02564103, 0.04262755, 0.16227633],
       [0.        , 0.        , 0.04299198, 0.14338194],
       [0.        , 0.        , 0.04262755, 0.12730503],
       [0.        , 0.        , 0.04225948, 0.11765486],
       [0.        , 0.        , 0.04225948, 0.1148414 ],
       [0.        , 0.        , 0.04225948, 0.11001833],
       [0.        , 0.        , 0.04225583, 0.10679893],
       [0.        , 0.        , 0.04116254, 0.10197585],
       [0.        , 0.        , 0.04262026, 0.09715278],
       [0.        , 0.        , 0.04225583, 0.09312953],
       [0.        , 0.        , 0.04115889, 0.08830646]])

In [ ]:
test_X[0,:,:]

array([[0.        , 0.        , 0.03534985, 0.02433643],
       [0.        , 0.        , 0.03425656, 0.02433643],
       [0.        , 0.        , 0.03571429, 0.02473835],
       [0.        , 0.        , 0.03498542, 0.02433643],
       [0.02941176, 0.01282051, 0.03389213, 0.02433643],
       [0.        , 0.02564103, 0.03753644, 0.02433643],
       [0.        , 0.        , 0.0393586 , 0.02433643],
       [0.06862745, 0.08974359, 0.04154519, 0.02473835],
       [0.05882353, 0.21794872, 0.0462828 , 0.02634604],
       [0.02941176, 0.5       , 0.05612245, 0.02956142],
       [0.01960784, 0.01282051, 0.06450437, 0.03116911],
       [0.        , 0.        , 0.05721574, 0.04362872]])

In [ ]:
y_train = train_y.reshape((-1,1))

In [ ]:
# reshape input to be 3D [samples, timesteps, features]

print( train_X.shape, train_y.shape, test_X.shape, test_y.shape)
 


(17072, 12, 4) (17072,) (4269, 12, 4) (4269,)


Training and evaluating models 

In [ ]:
#Metrics to monitor 
metrics = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),

      #keras.metrics.AUC(name='auc'),
      #keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
#defining the hyperModel
#  input_shape= inputShape to be removed 
from kerastuner import HyperModel
class LSTMHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()

        model.add(
            LSTM(units=hp.Int('units_l1', 30, 44, 4, default=40), input_shape= input_shape, 
                recurrent_dropout= hp.Float('recurrent_dropout_1', min_value=0.05, max_value=0.2, default=0.1, step=0.05,) , return_sequences=True ))
           
        model.add(
            LSTM(units=hp.Int('units_l2', 10, 24, 4, default=20),
            
            recurrent_dropout= hp.Float('recurrent_dropout_2', min_value=0.05, max_value=0.2, default=0.1, step=0.1,)  ))
          
        model.add(Dense(1, activation='sigmoid'))

        hp_learning_rate = hp.Choice('learning_rate', values=[  1e-3, 1e-4])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics= metrics )
        
        return model

In [ ]:
input_shape= (train_X.shape[1], train_X.shape[2])
hypermodel = LSTMHyperModel(input_shape)


In [ ]:
input_shape

(12, 4)

In [ ]:
from kerastuner import BayesianOptimization

#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#stop_early,   validation_split=0.2,
#tensorboardk = keras.callbacks.TensorBoard(log_dir='/tmp/tb_logs', histogram_freq= 1 , batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

tuner_bo = BayesianOptimization(
            hypermodel,
            objective= kt.Objective("recall", direction="max")  ,
            max_trials= 5,
            seed=7,
            executions_per_trial= 2  ,
            overwrite=True ,  
            directory="/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/15min"
            )
tuner_bo.search(train_X, y_train, epochs= 100 ,  batch_size = 64  , use_multiprocessing=True  , verbose=2 )
#shuffle=False,  

Trial 5 Complete [00h 20m 05s]
recall: 0.6239782273769379

Best recall So Far: 0.6335150003433228
Total elapsed time: 01h 37m 58s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Take best model 
best_model = tuner_bo.get_best_models(num_models=1)[0]
best_model.build(train_X.shape)

In [ ]:
best_model.evaluate( test_X, test_y, verbose=0)

[0.013319991528987885,
 0.9906283617019653,
 0.8430656790733337,
 0.5953608155250549]

In [ ]:
y_pred = best_model.predict( test_X,  batch_size= 128, verbose=1)

34/34 [==============================] - 1s 7ms/step


In [ ]:
y_pred =(y_pred>0.5)

# 15 min 

In [ ]:
best_model.get_config()
best_model.optimizer.get_config()
best_model.history.params
best_model.loss

In [ ]:
print (best_model.get_config()) 
print( best_model.optimizer.get_config()) 
print(best_model.history.params ) 
print ( best_modell.loss ) 

{'name': 'sequential_1', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 40, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 

In [ ]:
print (best_model.get_config()) 
print( best_model.optimizer.get_config()) 
print(best_model.history.params ) 
print ( best_model.loss ) 

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 38, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4249
         1.0       0.95      0.90      0.93        21

    accuracy                           1.00      4270
   macro avg       0.97      0.95      0.96      4270
weighted avg       1.00      1.00      1.00      4270



In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4249
         1.0       0.86      0.90      0.88        21

    accuracy                           1.00      4270
   macro avg       0.93      0.95      0.94      4270
weighted avg       1.00      1.00      1.00      4270



In [ ]:
best_model.save('/content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/15min')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/ThesisExperiments/FlashFloodPredictionBenchmarkLSTM/15min/assets


# 30 min

In [ ]:
print (best_model.get_config() ) 
print( best_model.optimizer.get_config())
print (best_model.history.params )
print ( best_model.loss ) 

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 42, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4249
         1.0       0.86      0.86      0.86        21

    accuracy                           1.00      4270
   macro avg       0.93      0.93      0.93      4270
weighted avg       1.00      1.00      1.00      4270



In [ ]:
best_model.save('')

# 45 min

In [ ]:
print ( best_model.get_config() )
print ( best_model.optimizer.get_config() )
print ( best_model.history.params )
print ( best_model.loss )

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 30, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4248
         1.0       0.90      0.86      0.88        21

    accuracy                           1.00      4269
   macro avg       0.95      0.93      0.94      4269
weighted avg       1.00      1.00      1.00      4269



In [ ]:
best_model.save('')

# 60 min

In [ ]:
print ( best_model.get_config() )
print ( best_model.optimizer.get_config() )
print ( best_model.history.params )
print ( best_model.loss )

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 42, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4248
         1.0       0.95      0.90      0.93        21

    accuracy                           1.00      4269
   macro avg       0.97      0.95      0.96      4269
weighted avg       1.00      1.00      1.00      4269



In [ ]:
best_model.save('')

# 75 min

In [ ]:
print ( best_model.get_config() )
print ( best_model.optimizer.get_config() )
print ( best_model.history.params )
print ( best_model.loss )

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 42, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4248
         1.0       0.94      0.76      0.84        21

    accuracy                           1.00      4269
   macro avg       0.97      0.88      0.92      4269
weighted avg       1.00      1.00      1.00      4269



In [ ]:
best_model.save('')

# 90 min

In [ ]:
print ( best_model.get_config() )
print ( best_model.optimizer.get_config() )
print ( best_model.history.params )
print ( best_model.loss )

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 42, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4248
         1.0       0.55      0.81      0.65        21

    accuracy                           1.00      4269
   macro avg       0.77      0.90      0.83      4269
weighted avg       1.00      1.00      1.00      4269



In [ ]:
best_model.save('')

# 105 min

In [ ]:
print ( best_model.get_config() )
print ( best_model.optimizer.get_config() )
print ( best_model.history.params )
print ( best_model.loss )

{'name': 'sequential', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}}, {'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'batch_input_shape': (None, 12, 4), 'dtype': 'float32', 'return_sequences': True, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 34, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'recurrent_initializer': {'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'unit_forget_bias': True, 'kernel_regularizer': None, 'recurrent_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'recurrent_constraint': None, 'bias_constraint': None, 'dropout': 0.

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4248
         1.0       0.82      0.43      0.56        21

    accuracy                           1.00      4269
   macro avg       0.91      0.71      0.78      4269
weighted avg       1.00      1.00      1.00      4269



In [ ]:
best_model.save('')

# 120 min

In [ ]:
print ( best_model.get_config() )
print ( best_model.optimizer.get_config() )
print ( best_model.history.params )
print ( best_model.loss )

In [ ]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6355
         1.0       0.94      0.90      0.92        50

    accuracy                           1.00      6405
   macro avg       0.97      0.95      0.96      6405
weighted avg       1.00      1.00      1.00      6405



In [ ]:
best_model.save('')